<a href="https://colab.research.google.com/github/ncmiquel/03MIAR---Algoritmos-de-Optimizacion/blob/main/Algoritmos_AG3_Ignacio_Carrillo_Miquel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AG3 - ACTIVIDAD GUIADA 3<br>**
Nombre: Ignacio Carrillo Miquel<br>
https://github.com/ncmiquel/03MIAR---Algoritmos-de-Optimizacion 

https://colab.research.google.com/drive/1zMWLBrapRsRCnlCWmNYKiAiMzA_k8vbl?usp=sharing


# PROBLEMA DEL AGENTE VIAJERO

Vamos a tratar de resolver el problema del agente viajero (TSP), en el que buscamos encontrar la ruta más corta que tenemos que hacer para visitar un conjunto determinado de ciudades y regresar a la ciudad de origen, visitando cada ciudad solo una vez.

Así que, antes de nada, vamos a instalar las librerías que vamos a necesitar para esta práctica:

In [22]:
!pip install requests    # Hacer llamadas http a paginas de la red
!pip install tsplib95    # Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Y también importaremos más librerías con las que trabajaremos:

In [23]:
import urllib.request # Hacer llamadas http a paginas de la red
import tsplib95       # Modulo para las instancias del problema del TSP
import math           # Modulo de funciones matematicas. Se usa para exp
import random         # Para generar valores aleatorios 

Con estas instrucciones importamos a Google Colab el fichero con el que vamos a trabajar.

In [24]:
# Descargamos el fichero de datos (Matriz de distancias)
file = "swiss42.tsp";
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     # Descomprime el fichero de datos 

Empezamos con el problema importando los datos del fichero Swiss42 (carga del problema):

In [25]:
# Carga de datos y generación de objeto problem
problem = tsplib95.load(file)

La función `get_weight` nos dice la distancia entre dos nodos, a partir de la intersección de los mismos:

In [26]:
# Nos dice la distancia del nodo 0 al nodo 5
problem.get_weight(0, 7)

37

Generamos una lista con el número de los nodos y aristas del problema:

In [27]:
# Nodos
Nodos = list(problem.get_nodes())

# Aristas
Aristas = list(problem.get_edges())

Crearemos una solución aleatoria para comenzar con el algoritmo. Para ello crearemos esta función, que recorre todos los nodos, y elige aleatoriamente uno que no haya elegido:

In [28]:
# Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

Esta función devuelve la distancia entre dos nodos, a y b:

In [29]:
# Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

Esta función calcula la distancia total de una ruta establecida, es decir, entre todos los nodos de una solución específica:

In [30]:
# Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1],  problem)
  return distancia_total + distancia(solucion[len(solucion)-1], solucion[0], problem)

Para probar estas funciones, creamos una variable `s_random` y comprobamos que podemos crear una solución aleatoria a partir de los 42 nodos iniciales (comenzando siempre por el 0), así como, calcular la distancia de dicha ruta:

In [31]:
# Creamos una solución aleatoria para probar los resultados obtenidos al azar
s_random = crear_solucion(Nodos)
print(distancia_total(s_random, problem), s_random)

5427 [0, 40, 25, 34, 41, 26, 20, 30, 6, 9, 12, 18, 7, 24, 14, 22, 31, 21, 5, 32, 2, 10, 28, 4, 17, 8, 19, 15, 3, 36, 23, 33, 11, 29, 39, 13, 16, 38, 27, 37, 1, 35]


Comenzamos el problema buscando una solución aleatoria. Una vez definida la función `busqueda_aleatoria`, en la que podemos determinar el número de iteraciones que deseamos, nos quedaremos con la mejor solución:

In [32]:
# BUSQUEDA ALEATORIA

def busqueda_aleatoria(problem, N):
  # N: es el nº de iteraciones
  # Nodos: nº de nodos en el problema
  Nodos = list(problem.get_nodes())
  
  # Inicializamos con un valor alto (infinito)
  mejor_solucion = []
  mejor_distancia = float('inf')
  
  # Criterio de parada: repetir N veces
  for i in range(N):
    # Genera una solucion aleatoria
    solucion = crear_solucion(Nodos)
    # Calcula el valor objetivo(distancia total)
    distancia = distancia_total(solucion, problem)
    
    # Compara con la mejor obtenida hasta ahora
    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
      
  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion
  
# Busqueda aleatoria con 100.000 iteraciones
solucion = busqueda_aleatoria(problem, 100000)

Mejor solución: [0, 25, 41, 23, 19, 1, 30, 39, 21, 20, 34, 17, 15, 35, 8, 18, 26, 13, 11, 9, 4, 38, 33, 36, 37, 3, 40, 24, 10, 12, 22, 29, 28, 5, 14, 6, 16, 31, 27, 32, 7, 2]
Distancia     : 3469


Estamos muy lejos de la mejor solución (hasta el momento) para este problema, que es 1273. Vamos a probar con otro algoritmo

## BÚSQUEDA LOCAL

Vamos a probar con otro algoritmo, el de Búsqueda local. Este algoritmo trata de encontrar la solución óptima de un problema a partir de una solución inicial, mediante la exploración de soluciones cercanas y la comparación con la solución actual. Si se encuentra una solución mejor, se reemplaza la solución actual por la nueva solución y se vuelve a explorar. Este proceso se repite hasta que no se encuentren soluciones mejores, o hasta que se alcance un criterio de parada. Por ello, haremos intercambios de nodos dos a dos, para ver si mejoramos esta solución.

Este algoritmo emplea un enfoque heurístico que no garantiza encontrar la solución global óptima, pero es eficaz en la resolución de muchos problemas prácticos. Para comenzar, definiremos una función `genera_vecina` que genera soluciones vecinas:



In [33]:
# Función para generar soluciones vecinas

def genera_vecina(solucion):
  # Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  # Se puede modificar para aplicar otros generadores distintos que 2-opt
  mejor_solucion = []
  mejor_distancia = float('inf')
  
  # Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      
      # Se genera una nueva solución intercambiando los dos nodos i,j:
      # (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  

      # Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

print("Distancia Solucion Incial:", distancia_total(solucion, problem))

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3469
Distancia Mejor Solucion Local: 3269


Se observa una mejora significativa en la distancia respecto de la solución aleatoria, aunque continuamos lejos de la mejor solución encontrada.

Para mejorar esta solución vamos a aplicar la función `genera_vecinas` repetidamente, aunque ahora no fijaremos un numero de iteraciones. El criterio de salida de la función va a ser ahora no encontrar una mejor solución:

In [34]:
# BÚSQUEDA LOCAL

#  Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem, solucion):
  mejor_solucion = []
    
  # Esta parte del código hace que siempre obtengamos el mismo resultado, ya que toma la variable solucion obtenida anteriormente
  solucion_referencia = solucion
  mejor_distancia = distancia_total(solucion_referencia, problem)
  # Contador para saber las iteraciones que hacemos
  iteracion = 0
  while(1):
    # Incrementamos el contador
    iteracion +=1
    # Obtenemos la mejor vecina
    vecina = genera_vecina(solucion_referencia)
    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
    # Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      # Guarda la mejor solución encontrada
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion", iteracion, "la mejor solución encontrada es:", mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
    solucion_referencia = vecina

sol = busqueda_local(problem, solucion)

En la iteracion 32 la mejor solución encontrada es: [0, 1, 6, 5, 26, 18, 9, 39, 22, 38, 34, 33, 20, 3, 4, 12, 11, 25, 10, 30, 32, 31, 35, 36, 17, 21, 24, 40, 23, 41, 8, 29, 28, 13, 19, 14, 16, 15, 37, 7, 27, 2]
Distancia     : 1895


Ahora sí. Vemos que aplicar iterativamente el algortimo de búsqueda local, mejora considerablemente la solución, que si bien aún está lejos del mejor valor encontrado para este problema, podemos considerar que se trata de una buena solución.

# AMPLIACIÓN PROBLEMA DEL AGENTE VIAJERO

Búsqueda local. Desventajas(intensifica pero no diversifica): escapar de máximos(mínimos) locales. 3 opciones:
- Modificar la estructura de entornos
búsqueda en entornos variables(*)
- Permitir movimientos peores respecto a la solución actual
búsqueda tabú, recocido simulado
- Volver a comenzar con otras soluciones iniciales
búsquedas multi-arranque

Si lo que queremos es trabajar con entornos variables, deberemos cambiar la función para generar una solución vecina. En el caso que hemos visto hace un momento hemos usado el método 2-opt, por lo que ahora definiremos una nueva función `genera_vecina_2`, en la cual incluiremos un método de generación en entornos variables.

En este caso hemos optado por un enfoque de *intercambio de segmentos*, en el que se intercambian dos segmentos de la solución actual para formar una nueva solución. La función recorre todos los tamaños de segmentos y todas las posiciones iniciales de los segmentos, intercambiándolos con una posición anterior en la lista para encontrar la vecina con la distancia total más corta. La función devuelve la mejor solución encontrada.

In [35]:
def genera_vecina_2(solucion):
  mejor_solucion = []
  mejor_distancia = float('inf')
  
  # Recorremos todos los tamaños de segmentos
  for tamaño_segmento in range(2, len(solucion)//2 + 1):
    # Recorremos todas las posiciones iniciales de los segmentos
    for i in range(len(solucion) - tamaño_segmento + 1):
      j = i + tamaño_segmento - 1
      
      # Intercambiamos el segmento con una posición anterior en la lista
      for k in range(i):
        vecina = solucion[:k] + solucion[i:j+1] + solucion[k:i] + solucion[j+1:]
        distancia_vecina = distancia_total(vecina, problem)
        if distancia_vecina < mejor_distancia:
          mejor_distancia = distancia_vecina
          mejor_solucion = vecina
          
  return mejor_solucion

In [36]:
# BÚSQUEDA LOCAL

#  Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem, solucion):
  mejor_solucion = []
  
  # Esta parte del código hace que siempre obtengamos el mismo resultado, ya que toma la variable solucion obtenida anteriormente
  solucion_referencia = solucion
  mejor_distancia = distancia_total(solucion_referencia, problem)
  # Contador para saber las iteraciones que hacemos
  iteracion = 0
  while(1):
    # Incrementamos el contador
    iteracion +=1
    # Obtenemos la mejor vecina
    vecina = genera_vecina_2(solucion_referencia)
    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
    # Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      # Guarda la mejor solución encontrada
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion", iteracion, "la mejor solución encontrada es:", mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
    
    solucion_referencia = vecina

sol = busqueda_local(problem, solucion)

En la iteracion 23 la mejor solución encontrada es: [0, 1, 6, 4, 3, 27, 32, 30, 29, 28, 8, 10, 18, 26, 5, 19, 13, 12, 11, 25, 41, 23, 9, 21, 40, 24, 39, 22, 38, 34, 33, 20, 35, 36, 31, 17, 37, 15, 16, 14, 7, 2]
Distancia     : 1450


# METAHEURÍSTICAS - RECOCIDO SIMULADO



El enfriamiento, o recocido, simulado es una técnica de optimización basada en un proceso análogo al enfriamiento físico. El proceso comienza con una solución aleatoria a una temperatura elevada, donde las soluciones peores son aceptadas con una probabilidad cierta. La temperatura se va reduciendo gradualmente a lo largo del tiempo, lo que hace que la probabilidad de aceptar soluciones peores disminuya. En el proceso se espera encontrar la solución óptima o una aproximación cercana a ella.

In [37]:
# SIMULATED ANNEALING

# Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
# Mejorable eligiendo otra forma de elegir una vecina.

def genera_vecina_aleatorio(solucion):    # TAL VEZ PARA MEJORAR SE DEBERIA INCIDIR AQUÍ. POR EJEMPLO ELEGIR ENTRE LOS 5 MEJORES RESULTADOS

  # Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample(range(1,len(solucion)), 2))
  
  # Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

Ahora introduciremos las funciones auxiliares para modelar la temperatura

In [38]:
#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() < math.exp(-1*d/T):
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.999 # Hay más funciones (mirar apuntes) para ir bajando la temperatura


Definimos la función de recocido simulado:

- En cada iteración se genera una vecina aleatoria.
- Calculamos el valor de la solucion.
- Vemos si es mejor, y si lo es, la guardamos.
- Por último, establecemos el control de si cambiamos o no: si la distancia de la vecina es peor, o si también la probabilidad de quedarnos con una peor solucion.

In [39]:
def recocido_simulado(problem, TEMPERATURA):
  # problem = datos del problema
  # T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)   # Este es el componente aleatorio
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = float('inf')
  
  N = 0
  while TEMPERATURA > .0001:
    N += 1
    #Genera una solución vecina
    vecina = genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    # Si la nueva vecina es mejor se cambia  
    # Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es", mejor_solucion)
  print("con una distancia total de", mejor_distancia)
  print("Iteraciones:", N)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)


La mejor solución encontrada es [0, 3, 27, 2, 28, 29, 30, 32, 7, 14, 1, 4, 6, 5, 26, 18, 12, 11, 25, 41, 23, 8, 10, 13, 19, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34, 38, 22, 39, 21, 24, 40, 9]
con una distancia total de 1570
Iteraciones: 25316


# AMPLIACIÓN: MEJORA EN LA GENERACIÓN DE LA PARTE ALEATORIA DE RECOCIDO SIMULADO

Vamos a tratar de mejorar la solución aportada por el método del recocido simulado, incidiendo en la generación de la parte aleatoria del algoritmo. En este caso, en lugar de generar una sola solución aleatoria y empezar con ella, hemos modificado la función `genera_vecina_aleatorio_1`, de manea que, ahora, el algoritmo genera un número determinado de soluciones aleatorias, y se queda con la que mejor resultado da. En este caso hemos dejado un valor de 10, aunque hemos probado diferentes combinaciones (5, 15, 20, etc.).

In [40]:
def genera_vecina_aleatorio_1(solucion):    # TAL VEZ PARA MEJORAR SE DEBERIA INCIDIR AQUÍ. POR EJEMPLO ELEGIR ENTRE LOS 5 MEJORES RESULTADOS

  mejor_distancia = float('inf')
  mejor_solucion = []
  # Se eligen dos nodos aleatoriamente
  for iteracion in range(10):
    i,j = sorted(random.sample(range(1,len(solucion)), 2))
    solucion_provisional = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
    distancia_referencia = distancia_total(solucion_provisional, problem)

    if distancia_referencia < mejor_distancia:
        mejor_solucion = solucion_provisional
        mejor_distancia = distancia_referencia

  # Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return mejor_solucion

print(genera_vecina_aleatorio(solucion))


[0, 25, 41, 23, 19, 1, 30, 39, 21, 20, 34, 17, 15, 35, 8, 18, 26, 13, 11, 9, 4, 38, 33, 36, 37, 3, 29, 24, 10, 12, 22, 40, 28, 5, 14, 6, 16, 31, 27, 32, 7, 2]


In [41]:
def recocido_simulado(problem, TEMPERATURA):
  # problem: datos del problema
  # T: Temperatura
  
  solucion_referencia = crear_solucion(Nodos)   # Este es el componente aleatorio
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = float('inf')
  
  N = 0
  while TEMPERATURA > .00001:
    N += 1
    # Genera una solución vecina
    vecina = genera_vecina_aleatorio_1(solucion_referencia)
    
    # Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    # Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    # Si la nueva vecina es mejor se cambia  
    # Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es", mejor_solucion)
  print("con una distancia total de", mejor_distancia)
  print("Iteraciones:", N)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 1, 3, 4, 6, 5, 13, 19, 14, 16, 15, 37, 7, 26, 18, 12, 11, 25, 10, 29, 30, 34, 20, 33, 38, 22, 39, 21, 24, 40, 23, 41, 9, 8, 28, 2, 27, 32, 31, 35, 36, 17]
con una distancia total de 1514
Iteraciones: 27618


Después de probar diferentes soluciones al aplicar de nuevo la función `recocido_simulado` con estas nuevas condiciones, obtenemos mejores resultados que antes. Es frecuente que las mejores soluciones que quedan después de aplicar el algoritmo, esten por debajo de 1400, llegando a una mejor solución encontrada de 1318, muy cerca de 1273 (solo distan 45 km).

Cabe destacar que también hemos probado diferentes valores de enfriamiento (0.999·T, 0.9999·T, etc), comprobando que un tiempo de enfriamiento más lento favorece soluciones óptimas, aunque a costa de tiempo de ejecución (hemos llegado a ejecuciones de más de 12 minutos).

La mejor solución que hemos encontrado, la podemos comprobar con la función `distancia_total`:

In [42]:
# Esta es la mejor solución que hemos encontrado
print(distancia_total([0, 1, 6, 26, 18, 12, 11, 25, 10, 8, 9, 41, 23, 40, 24, 21, 39, 22, 38, 30, 29, 28, 32, 34, 33, 20, 35, 36, 31, 17, 7, 37, 15, 16, 14, 19, 13, 5, 4, 3, 2, 27], problem))

1318
